In [ ]:
#@title # Установка (5 мин без претрейнов)
%cd /content

!git clone https://github.com/noblebarkrr/mvsepless -b alpha_hf
# !curl -L https://github.com/noblebarkrr/vbach/archive/refs/heads/main.tar.gz | tar -xz --strip-components=1 -C /content/mvsepless
!pip install gradio
!pip install -r mvsepless/requirements.txt
!pip install https://github.com/Bebra777228/TrainVocModel-EN/releases/download/fixed-packages/fairseq_fixed-0.13.0-cp311-cp311-linux_x86_64.whl
!wget -O mvsepless/models/medley_vox/pretrained_models/xlsr_53_56k.pt https://dl.fbaipublicfiles.com/fairseq/wav2vec/xlsr_53_56k.pt
!cd mvsepless && python download_models.py

# NON-CLI

In [ ]:
# @title Mvsepless

%cd /content/mvsepless
!python multi_infer.py -gr

In [ ]:
# @title Mvsepless & Vbach

%cd /content/mvsepless
!python multi_infer.py -grvc

# CLI

In [ ]:
# @title Объединить аудио файлы в ансамбль

import os
import numpy

%cd /content/mvsepless

from ensem import ensemble_audio_files
from infer_utils.audio_processing.invert import invert_and_overlay_wav


input = "" # @param {"type":"string","placeholder":"Введите путь к папке с результатами моделей"}
input_orig = "" # @param {"type":"string","placeholder":"Введите путь к оригинальному аудио"}
output = "" # @param {"type":"string","placeholder":"Введите путь к папке сохранения результатов"}

type = "max_fft" # @param ["max_fft","min_fft","median_fft","max_wave","avg_fft"]

# Создаем выходную директорию, если ее нет
os.makedirs(output, exist_ok=True)

# Проверяем существование входной директории
if not os.path.exists(input):
    raise FileNotFoundError(f"Input directory {input} does not exist")

# Получаем список файлов
temp_ensem_files = [os.path.abspath(os.path.join(input, f))
         for f in os.listdir(input)
         if os.path.isfile(os.path.join(input, f))]

if not temp_ensem_files:
    raise ValueError("No files found in input directory")

weight_value = 1.0 # @param {"type":"number"}
weights = [weight_value] * len(temp_ensem_files)

output_file = os.path.join(output, "output.wav")
inverted_file = os.path.join(output, "inverted.wav")

ensemble_audio_files(files=temp_ensem_files, output=output_file, ensemble_type=type, weights=weights)
if input_orig != "":
    invert_and_overlay_wav(output_file, input_orig, inverted_file)

## Разделить аудио

In [ ]:
# @title Получить список моделей

%cd /content/mvsepless

model_type = "mel_band_roformer" # @param ["mel_band_roformer","bs_roformer","mdx23c","vr_arch","htdemucs","scnet","mdx_net","medley_vox"]

from model_info import model_info
model_info(model_type)

In [ ]:
# @title Инференс

%cd /content/mvsepless

#@markdown <b>Чтобы получить информацию о моделях, запустите ячейку 'Получить список моделей' с нужным типом модели</b>

template = "NAME_MODEL_STEM" # @param {"type":"string","placeholder":"Шаблон имени стемов, пример: NAME_MODEL_STEM"}

model_type = "mel_band_roformer" # @param ["mel_band_roformer","bs_roformer","mdx23c","vr_arch","htdemucs","scnet","mdx_net","medley_vox"]
model_name = "" # @param {"type":"string","placeholder":"Введите имя модели, которое есть в списке моделей (Можно получить список используя ячейку выше)"}
input = "" # @param {"type":"string","placeholder":"Введите путь к аудиофайлу/папке с аудиофайлами"}
output = "" # @param {"type":"string","placeholder":"Введите путь к папке сохранения стемов"}

output_format = "flac" # @param ["wav","mp3","flac"]
batch = False # @param {"type":"boolean"}
tta = False
select_stems = "" # @param {"type":"string","placeholder":"Введите стемы, которые хотите сохранить, пример: 'vocals drums' или 'male female karaoke''"}

#@markdown <details>
#@markdown <summary><b><u>Описание настроек</u></b></summary>
#@markdown
#@markdown > * <b><u>template</u></b> - Шаблон для именования результатов разделения.
#@markdown >
#@markdown >  Существуют три ключа:
#@markdown >
#@markdown >  > NAME - название оригинального аудио файла (без расширения)
#@markdown >
#@markdown >  > MODEL - название модели, использованной для разделения на стемы
#@markdown >
#@markdown >  > STEM - название стема
#@markdown >
#@markdown >  Пример:
#@markdown >
#@markdown >  NAME_STEM --> вход_vocals.mp3
#@markdown >
#@markdown >  > NAME - 'вход'
#@markdown >
#@markdown >  > STEM - vocals
#@markdown >
#@markdown >  MODEL_STEM --> mel_band_roformer_aname_4_stems_large_drums.mp3
#@markdown >
#@markdown >  > MODEL - 'mel_band_roformer_aname_4_stems_large'
#@markdown >
#@markdown >  > STEM - drums
#@markdown >
#@markdown >  mvsepless_STEM --> mvsepless_karaoke.mp3
#@markdown >
#@markdown >  > mvsepless - mvsepless (кастомное имя)
#@markdown >
#@markdown >  > STEM - karaoke
#@markdown >
#@markdown >  <b>Не работает с моделями на архитектуре Medley-Vox</b>
#@markdown
#@markdown > * <b><u>batch</u></b> - Пакетная обработка. Включается при batch = True
#@markdown >
#@markdown >  (В нашем случае если поставлена галочка напротив batch)
#@markdown >
#@markdown >  Включено - обрабатывается папка с аудиофайлами
#@markdown >
#@markdown >  Выключено - обрабатывается один аудиофайл
#@markdown
#@markdown > * <b><u>select_stems</u></b> - Выбор стемов для разделения
#@markdown >
#@markdown >  Не работает с:
#@markdown >
#@markdown >  > моделями на архитектуре Medley-Vox
#@markdown >
#@markdown >  > моделями с целевым инструментом (Target Instrument is not "No")
#@markdown >
#@markdown >  > моделями на архитектуре MDX-NET и VR ARCH, если выбрано больше одного стема
#@markdown
#@markdown > * <b><u>model_type</u></b> - Тип модели, то есть её архитектура
#@markdown
#@markdown > * <b><u>model_name</u></b> - Название модели для разделения вокала

run_command = f'python multi_infer.py -i "{input}" -o {output} -of {output_format}  {("--use_tta" if tta else "")} {("--batch" if batch else "")} -inst --model_type {model_type} --model_name {model_name} {(f"--select {select_stems}" if select_stems != "" else "")} --template {template}'
!$run_command

## Замена вокала

In [ ]:

%cd /content/mvsepless

import os
from datetime import datetime
from rvc.scripts.voice_conversion import voice_pipeline


def voice_conversion(input, model, pitch, ir, fr, rms, f0, hop, prtct, of, f0_min, f0_max, output, template, batch):
      if batch:
          for filename in os.listdir(input):
              file = os.path.join(input, filename)
              if os.path.isfile(file):
                  file_name = os.path.basename(file)
                  namefile = os.path.splitext(file_name)[0]
                  time_create_file = datetime.now().strftime("%Y%m%d_%H%M%S")
                  output_name = (
                      template
                      .replace("DATETIME", time_create_file)
                      .replace("NAME", namefile)
                      .replace("MODEL", model)
                      .replace("F0METHOD", f0)
                      .replace("PITCH", f"{pitch}")
                  )
                  voice_pipeline(file, model, pitch, ir, fr, rms, f0, hop, prtct, of, f0_min, f0_max, output, output_name)
      else:
          time_create_file = datetime.now().strftime("%Y%m%d_%H%M%S")
          output_name = (
              template
              .replace("DATETIME", time_create_file)
              .replace("NAME", namefile)
              .replace("MODEL", model)
              .replace("F0METHOD", f0)
              .replace("PITCH", pitch)
          )
          voice_pipeline(input, model, pitch, ir, fr, rms, f0, hop, prtct, of, f0_min, f0_max, output, output_name)


song_input = ""  # @param {type:"string"}
model_name = ""  # @param {type:"string"}
batch = False # @param {"type":"boolean"}

# @markdown #### Основные настройки
pitch = 0  # @param {type:"slider", min:-48, max:48, step:12}
index_rate = 0  # @param {type:"slider", min:0, max:1, step:0.01}
filter_radius = 3  # @param {type:"slider", min:0, max:7, step:1}
volume_envelope = 0.25  # @param {type:"slider", min:0, max:1, step:0.01}

# @markdown #### Настройки F0
method = "rmvpe+"  # @param ["rmvpe+", "mangio-crepe", "fcpe"]
hop_length = 128  # @param {type:"slider", min:32, max:512, step:16}
protect = 0.33  # @param {type:"slider", min:0, max:0.5, step:0.01}
f0_min = 50  # @param {type:"slider", min:0, max:500, step:1}
f0_max = 1100  # @param {type:"slider", min:100, max:2000, step:10}

# @markdown #### Выходные настройки
output_format = "mp3"  # @param ["mp3", "wav", "flac"]
output_path = ""  # @param {type:"string"}

# Задаем шаблон вручную (можно менять порядок и состав)
template = "DATETIME_NAME_PITCH"  # @param {type:"string"}

voice_conversion(song_input, model_name, pitch, index_rate, filter_radius, volume_envelope, method, hop_length, protect, output_format, f0_min, f0_max, output_path, template, batch)